In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
file_path = '/content/drive/My Drive/DataSets/images.zip'

import zipfile

zip_file_path = file_path
extraction_path = '/content/FaceImages/'

import os
os.makedirs(extraction_path, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

print(f'Files extracted to {extraction_path}')

Files extracted to /content/FaceImages/


In [5]:
!pip install keras_preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.6 MB/s eta 0:00:00


In [6]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [7]:
TRAIN_DIR = 'FaceImages/images/train'
TEST_DIR = 'FaceImages/images/validation'

In [8]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels


In [9]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

surprise completed
sad completed
happy completed
angry completed
neutral completed
fear completed
disgust completed


In [10]:
print(train)

                                            image     label
0      FaceImages/images/train/surprise/32370.jpg  surprise
1      FaceImages/images/train/surprise/20413.jpg  surprise
2       FaceImages/images/train/surprise/8605.jpg  surprise
3      FaceImages/images/train/surprise/21633.jpg  surprise
4      FaceImages/images/train/surprise/19630.jpg  surprise
...                                           ...       ...
28816   FaceImages/images/train/disgust/12115.jpg   disgust
28817   FaceImages/images/train/disgust/13458.jpg   disgust
28818   FaceImages/images/train/disgust/24431.jpg   disgust
28819   FaceImages/images/train/disgust/15287.jpg   disgust
28820   FaceImages/images/train/disgust/12838.jpg   disgust

[28821 rows x 2 columns]


In [11]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

surprise completed
sad completed
happy completed
angry completed
neutral completed
fear completed
disgust completed


In [12]:
print(test)
print(test['image'])

                                                image     label
0     FaceImages/images/validation/surprise/27070.jpg  surprise
1     FaceImages/images/validation/surprise/29954.jpg  surprise
2     FaceImages/images/validation/surprise/25906.jpg  surprise
3     FaceImages/images/validation/surprise/15635.jpg  surprise
4     FaceImages/images/validation/surprise/15101.jpg  surprise
...                                               ...       ...
7061   FaceImages/images/validation/disgust/33209.jpg   disgust
7062   FaceImages/images/validation/disgust/14954.jpg   disgust
7063   FaceImages/images/validation/disgust/16264.jpg   disgust
7064    FaceImages/images/validation/disgust/2732.jpg   disgust
7065    FaceImages/images/validation/disgust/4737.jpg   disgust

[7066 rows x 2 columns]
0       FaceImages/images/validation/surprise/27070.jpg
1       FaceImages/images/validation/surprise/29954.jpg
2       FaceImages/images/validation/surprise/25906.jpg
3       FaceImages/images/validation/su

In [13]:
from tqdm.notebook import tqdm

In [14]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features


In [15]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [16]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [17]:
x_train = train_features/255.0
x_test = test_features/255.0

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [20]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [21]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [24]:
!pip install tensorflow
# Import the required module.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data augmentation for training data
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Apply augmentation to training data
datagen.fit(x_train)

In [25]:
from keras.regularizers import l2

model = Sequential()
# Convolutional layers with L2 regularization
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1), kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# Fully connected layers with L2 regularization
model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.3))
# Output layer
model.add(Dense(7, activation='softmax'))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
from keras.layers import BatchNormalization

model = Sequential()
# Add batch normalization after Conv layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# Fully connected layers with batch normalization
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
# Output layer
model.add(Dense(7, activation='softmax'))


In [27]:
from keras.optimizers import RMSprop

# Compile model using RMSprop optimizer
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [30]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping # Import the EarlyStopping callback

# Reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    min_lr=0.00001
)

# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Add the callbacks to model.fit
model.fit(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 35s 95ms/step - accuracy: 0.1637 - loss: 2.7129 - val_accuracy: 0.2588 - val_loss: 2.0012 - learning_rate: 1.0000e-04
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 12s 51ms/step - accuracy: 0.1979 - loss: 2.3523 - val_accuracy: 0.2352 - val_loss: 1.9145 - learning_rate: 1.0000e-04
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 21s 52ms/step - accuracy: 0.2137 - loss: 2.2095 - val_accuracy: 0.2705 - val_loss: 1.8846 - learning_rate: 1.0000e-04
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - accuracy: 0.2385 - loss: 2.0763 - val_accuracy: 0.3382 - val_loss: 1.6951 - learning_rate: 1.0000e-04
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 21s 53ms/step - accuracy: 0.2750 - loss: 1.9671 - val_accuracy: 0.3579 - val_loss: 1.6531 - learning_rate: 1.0000e-04
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step - accuracy: 0.3078 - loss: 1.8710 - val_accuracy: 0.3939 - val_loss: 1.6086 - learning_rate: 1.0000e-04
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 12s 5

In [33]:
model_json = model.to_json()
with open('/content/drive/My Drive/DataSets/emotiondetector.json', 'w') as json_file:
    json_file.write(model_json)
model.save("/content/drive/My Drive/DataSets/emotiondetector.h5")